In [1]:
import numpy as np
from time import time

import sys

def b2d(a):
        b = 0
        for q in range(len(a)):
                b+=(2**(len(a)-1-q))*a[q]
        return b


from keras.datasets import mnist

from tmu.models.classification.vanilla_classifier import TMClassifier

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

#X_test data into image 1568 elements in 10,000 array catch randomn 8 arrays
#needed 32 bits in each array   1568/32=49;



X_train = np.where(X_train.reshape((X_train.shape[0], 28*28)) > 75, 1, 0)
X_test = np.where(X_test.reshape((X_test.shape[0], 28*28)) > 75, 1, 0)

tm = TMClassifier(10000,25, 5.5, max_included_literals=8, platform='CPU', weighted_clauses=True,incremental='True')

#print(X_test[0])
#print(X_test)

temp=np.empty(1568,dtype=int)

final=[[0]*49]*8

dum=0
dectemp=np.empty(32,dtype=int)

ref_num=np.random.randint(0,1576,size=8)
for m in range(8):
        for l in range(784):
                temp[l]=X_test[ref_num[m]][l]
                temp[l+784]=1^X_test[ref_num[m]][l]
        for n in range(49):
                dectemp=temp[n*32:n*32+32]
                dec=b2d(dectemp)
                final[m][n]=dec


print(ref_num[0])
print(final[0][:])

with open("final.h",mode="w") as file:
        my_data = f"uint32_t final[8][49] = {str(final[:][:])};"
        my_data = my_data.replace("[[","{{")
        my_data = my_data.replace("]]","}}")
        my_data = my_data.replace("], [","}, {")
        file.write(my_data)

print("\nAccuracy over 8 epochs:\n")
for i in range(8):
        start_training = time()
        tm.fit(X_train, Y_train)
        stop_training = time()

        start_testing = time()
        result = 100*(tm.predict(X_test) == Y_test).mean()
        stop_testing = time()

        print("#%d Accuracy: %.2f%% Training: %.2fs Testing: %.2fs" % (i+1, result, stop_training-start_training, stop_testing-start_testing))
        
        total_class=0
        
        for i in range (10):
                total_clause=0
                for j in range (500):
                        k=tm.number_of_include_actions(i,j)
                        total_clause=total_clause+k
                total_class=total_class+total_clause       
                
        print("Total Includes: %d \n" % total_class) 
         
        


1529
[0, 0, 0, 0, 16, 896, 12288, 229376, 3145728, 50331648, 805306371, 48, 768, 12288, 196608, 7340032, 117440512, 1879048198, 96, 1536, 24576, 262144, 0, 0, 65535, 4294967295, 4294967295, 4294967295, 4294967295, 4293918719, 4236247039, 3489660924, 2147483599, 4294966527, 4294955007, 4294770687, 4291821567, 4244635647, 3489660924, 4294967183, 4294965503, 4294938623, 4294574079, 4288675839, 4194303999, 2684354555, 4294967295, 4294967295, 4294967295]

Accuracy over 8 epochs:

2023-12-09 10:17:57,116 - tmu.models.classification.vanilla_classifier - DEBUG - Initializing clause bank....
#1 Accuracy: 87.31% Training: 269.42s Testing: 6.27s
Total Includes: 7508 

#2 Accuracy: 92.64% Training: 205.68s Testing: 6.08s
Total Includes: 7565 

#3 Accuracy: 90.96% Training: 215.46s Testing: 9.80s
Total Includes: 7118 

#4 Accuracy: 92.62% Training: 223.61s Testing: 7.49s
Total Includes: 7413 

#5 Accuracy: 93.33% Training: 185.32s Testing: 6.20s
Total Includes: 7135 

#6 Accuracy: 93.55% Training: 